In [1]:
import pandas as pd
import numpy as np
from numpy import concatenate
from pandas import concat
from pandas import read_csv
from pandas import DataFrame
import tensorflow as tf
from tensorflow import keras
import os
import datetime
import IPython
import IPython.display
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import explained_variance_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Flatten, LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Bidirectional, Input, Reshape, RepeatVector, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
%matplotlib inline
from scipy.stats import randint as sp_randint
np.random.seed(123) # for reproducibility
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier, KerasRegressor
import keras_tuner

In [2]:
# CARGAR LA BASE DE DATOS
from pandas import read_csv
datos = read_csv('Bucaramanga_Lags.csv', encoding='latin-1', sep=";")
# Dropping the id and date columns
datos.columns

Index(['Municipio', 'Año', 'Semana_Epi', 'Casos_Dengue', 'Prom_Dia_Preci',
       'Prom_Dia_Preci_1', 'Prom_Dia_Preci_2', 'Prom_Dia_Preci_3',
       'Prom_Dia_Preci_4', 'Prom_Dia_Preci_5', 'Prom_Dia_Preci_6',
       'Total_Preci_Sem', 'Total_Preci_Sem_1', 'Total_Preci_Sem_2',
       'Total_Preci_Sem_3', 'Total_Preci_Sem_4', 'Total_Preci_Sem_5',
       'Total_Preci_Sem_6'],
      dtype='object')

In [3]:
datos = datos.drop(['Municipio', 'Año', 'Semana_Epi','Prom_Dia_Preci_1', 'Prom_Dia_Preci_2', 'Prom_Dia_Preci_3',
       'Prom_Dia_Preci_4', 'Prom_Dia_Preci_5', 'Prom_Dia_Preci_6', 'Total_Preci_Sem_1', 'Total_Preci_Sem_2',
       'Total_Preci_Sem_3', 'Total_Preci_Sem_4', 'Total_Preci_Sem_5','Total_Preci_Sem_6'],axis=1)
datos.columns

Index(['Casos_Dengue', 'Prom_Dia_Preci', 'Total_Preci_Sem'], dtype='object')

In [4]:
train_size = int(len(datos) * 0.7)
test_size = len(datos) - train_size
train, test = datos.iloc[0:train_size], datos.iloc[train_size:len(datos)]
print(len(train), len(test))

436 188


In [5]:
f_columns = ['Prom_Dia_Preci', 'Total_Preci_Sem']
f_transformer = MinMaxScaler(feature_range=(0, 1))
f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())

In [6]:
dengue_transformer = MinMaxScaler(feature_range=(0, 1))
dengue_transformer = dengue_transformer.fit(train[['Casos_Dengue']])
train['Casos_Dengue'] = dengue_transformer.transform(train[['Casos_Dengue']])
test['Casos_Dengue'] = dengue_transformer.transform(test[['Casos_Dengue']])

In [7]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [8]:
time_steps = 10
# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(train, train.Casos_Dengue, time_steps)
X_test, y_test = create_dataset(test, test.Casos_Dengue, time_steps)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape )

(426, 10, 3) (426,) (178, 10, 3) (178,)


In [9]:
def cnn_model(dropout_rate=0.2, optimizer="adam", activation="relu", No_of_CONV_and_Maxpool_layers=3,
              No_of_Units_in_dense_layers=100, padding ='same', Conv1d_filters=60):

    
    #model sequential
    cnn_model=Sequential()
    
    #CAPA DE ENTRADA
    cnn_model.add(keras.layers.Conv1D(32, kernel_size=2, strides=1,
                                      activation=activation, input_shape=(X_train.shape[1], X_train.shape[2]),padding=padding))
    
    
    for i in range(No_of_CONV_and_Maxpool_layers):
        
        cnn_model.add(keras.layers.Conv1D(Conv1d_filters, kernel_size=2, strides=1, activation=activation, padding=padding))
        cnn_model.add(keras.layers.MaxPooling1D(pool_size=2, padding=padding))
        
    cnn_model.add(Dropout(dropout_rate))
    cnn_model.add(Flatten())
    
    
    
    cnn_model.add(keras.layers.Dense(units=No_of_Units_in_dense_layers, activation=activation))
    cnn_model.add(Dropout(dropout_rate)) 
                  
    cnn_model.add(Dense(1))
    
    
    #COMPILACIÓN DEL MODELO
    cnn_model.compile(optimizer=optimizer, loss = 'mse', metrics = ['mse'])
    
    

    return cnn_model

In [10]:
#HIPERPARÁMETROS

activation=['relu','tanh', 'sigmoid', 'linear']
dropout_rate=[0.1, 0.3, 0.5]
No_of_CONV_and_Maxpool_layers=[2,3,4]
Conv1d_filters=[60,40,80,120,220]
No_of_Units_in_dense_layers=[128,64,32,256]
optimizer = ['adam', 'sgd', 'rmsprop']
padding = ['valid', 'same', 'causal']

param_grid = dict(model__activation=activation,
                  model__dropout_rate=dropout_rate,
                  model__padding=padding,
                  model__No_of_CONV_and_Maxpool_layers=No_of_CONV_and_Maxpool_layers,
                  model__Conv1d_filters=Conv1d_filters,
                  model__No_of_Units_in_dense_layers=No_of_Units_in_dense_layers,
                  model__optimizer=optimizer,)

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer

model =  KerasRegressor(model=cnn_model, verbose=0)

rs = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring='neg_mean_squared_error',
                        n_jobs=-1, cv=KFold(3), n_iter=20, verbose=0,random_state = 123, return_train_score = True)

In [12]:
fitted_model = rs.fit(X_train, y_train)

In [13]:
resultados = pd.DataFrame(fitted_model.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False)\
    .head(10)

,param_model__padding,param_model__optimizer,param_model__dropout_rate,param_model__activation,param_model__No_of_Units_in_dense_layers,param_model__No_of_CONV_and_Maxpool_layers,param_model__Conv1d_filters,mean_test_score,std_test_score,mean_train_score,std_train_score
1,same,rmsprop,0.5,linear,256,3,80,-0.003970,0.001635,-0.003435,0.000817
4,same,rmsprop,0.1,tanh,128,2,220,-0.004571,0.001872,-0.004789,0.002070
10,valid,rmsprop,0.3,linear,128,2,60,-0.009147,0.006014,-0.007467,0.001932
9,same,rmsprop,0.5,relu,32,3,220,-0.011156,0.006941,-0.007241,0.001083
8,valid,adam,0.1,linear,32,2,40,-0.012662,0.004025,-0.011692,0.004551
18,same,rmsprop,0.5,relu,128,4,80,-0.020954,0.012024,-0.011660,0.005447
6,same,sgd,0.3,linear,64,4,220,-0.026837,0.018134,-0.022362,0.005821
12,valid,sgd,0.3,tanh,32,2,120,-0.027046,0.012712,-0.023756,0.008228
3,same,adam,0.3,sigmoid,64,2,120,-0.034871,0.025960,-0.036406,0.012101
19,same,sgd,0.3,sigmoid,128,2,40,-0.046347,0.026919,-0.036724,0.011570


In [14]:
print("Best: %f using %s" % (fitted_model.best_score_, fitted_model.best_params_))

Best: -0.003970 using {'model__padding': 'same', 'model__optimizer': 'rmsprop', 'model__dropout_rate': 0.5, 'model__activation': 'linear', 'model__No_of_Units_in_dense_layers': 256, 'model__No_of_CONV_and_Maxpool_layers': 3, 'model__Conv1d_filters': 80}
